In [ ]:
!pip3 install openpyxl
!pip3 install weasyprint
!pip3 install gizeh
!pip3 install pycairo
!pip3 install cairocffi
!pip3 install more_itertools 

In [42]:
import gizeh as gz
import json
import numpy as np
import pandas as pd
import glob
from more_itertools.recipes import grouper, pairwise
from PIL import Image


print("Jupyter online")

Jupyter online


In [86]:
#Preprocessing

#Openpose keypointdata extractor

#https://sduxbury.medium.com/how-you-can-build-practical-applications-by-quantifying-observations-from-video-e266b945eea0
#Point sets are in the form [coordinate, coordinate, confidence]

In [28]:
def open_json(jsonfile):
    jsonopen = open(jsonfile)
    jfile = json.load(jsonopen)
    jsonopen.close()
    x = pd.DataFrame(jfile)
    return x

def framenames(path, fps):
    frames = np.arange(0, 900*fps+1, 24).tolist()
    startname = 'frame'
    endname = '_keypoints.json'
    frames = frames[1:]
    framelist = []
    for i in frames:
        framelist.append(path+startname+str(i)+endname)
    return(framelist)

def data_loader(framelist):
    pose1= []
    pose2= []
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    count5 = 0
    for jsonfile in framelist:
        data = open_json(jsonfile)
        if len(data.index) == 2:
            s1 = data.iloc[0,:]
            s2 = data.iloc[1,:]
            count1 += 1
        if len(data.index) == 3:
            s1 = data.iloc[0,:]
            s2 = data.iloc[2,:]
            count2 += 1
        if len(data.index) == 1:
            s1 = data.iloc[0,:]
            s2 = data.iloc[1,:]
            count3 += 1
        if len(data.index) == 4:        #NEEDS WORK, KN5 frame 216 has 4 skeletons in it
            s1 = data.iloc[0,:]
            s2 = data.iloc[1,:]
            count4 += 1
        if len(data.index) == 5:
            s1 = data.iloc[0,:]
            s2 = data.iloc[1,:]
            count5 += 1

        sig1 = s1["people"]
        sig2 = s2["people"]

        signer1poses = pd.DataFrame.from_dict(sig1, orient='index')
        signer2poses = pd.DataFrame.from_dict(sig2, orient='index')

        signer1 = signer1poses.iloc[1:5,:]
        signer2 = signer2poses.iloc[1:5,:]

        signer1 = signer1.drop(["face_keypoints_2d"])
        signer2 = signer2.drop(["face_keypoints_2d"])
        
        pose1.append(signer1)
        pose2.append(signer2)
    poses = pose1 + pose2
    print('There are '+str(count3)+' frames with 1 person in them')
    print('There are '+str(count1)+' frames with 2 persons in them')
    print('There are '+str(count2)+' frames with 3 persons in them')
    print('There are '+str(count4)+' frames with 4 persons in them')
    print('There are '+str(count5)+' frames with 5 persons in them')
    return poses

def scale(lst):
    newdata = []
    for i,d in enumerate(lst):
        if d.any() > 0:
            newdata.append(d/500)
        else:
            newdata.append(d)
    return newdata

def normalizer(lst,nmbr):
    fin = []
    norm = (nmbr*1.8)
    #assuming person 2 always sits on the right in the frame which gives him/her different y coordinates but similar x coordinates.
    z = np.array([norm,0])
    #normalize the y coordinate appropriately for person 1 so relative distances stay the same. 
    for i,d in enumerate(lst):
        if d.any() > 0:
            d = d-z
            fin.append(d)
        else:
            fin.append(d) 
    return fin

def pose_processor(poses):                                                      #Finish pose processor
    poses = poses
    lengthp = len(poses)
    middle = lengthp/2
    p1 = poses[:int(middle)]
    p2 = poses[int(middle):]

    #Create confidenceintervals index
    confidenceintervals = np.arange(-1, 75, 3).tolist()
    confidenceintervals = confidenceintervals[1:]
    #print(confidenceintervals)

    #Create x and y coordinates index
    coordinates = np.arange(0, 75, 1).tolist()
    cs1 = set(confidenceintervals)
    coorindex = [x for x in coordinates if x not in cs1]
    #print(coorindex)

    person1 = []
    for framepose in p1:
        signer1 = pd.DataFrame(framepose) 
        skel1 = signer1.iloc[0,:]
        #Get confidence intervals for signer 1 and 2 per keypoint
        ci1 = [skel1[i] for i in confidenceintervals]
        #Get x and y coordinates for signer 1 and 2 per keypoint
        skelcoor1 = [skel1[i] for i in coorindex]
        sk1 = np.array_split(skelcoor1, 25)
        #Normalize on neck Y coordinate as neck position
        neck2 = sk1[1]
        newdata1 = scale(normalizer(sk1, neck2[0]))
        person1.append(newdata1)
        
    person2 = []
    for framepose in p2:
        signer2 = pd.DataFrame(framepose) 
        skel2 = signer2.iloc[0,:]
        #Get confidence intervals for signer 1 and 2 per keypoint
        ci2 = [skel2[i] for i in confidenceintervals]
        #Get x and y coordinates for signer 1 and 2 per keypoint
        skelcoor2 = [skel2[i] for i in coorindex]
        sk2 = np.array_split(skelcoor2, 25)

        newdata2 = scale(sk2)
        person2.append(newdata2)
    #return sk1, sk2
    signers = person1 + person2
    return signers

def hand_processor(poses):
    poses = poses
    lengthp = len(poses)
    middle = lengthp/2
    p1 = poses[:int(middle)]
    p2 = poses[int(middle):]

    #Create confidenceintervals index
    confidenceintervalh = np.arange(-1, 63, 3).tolist()
    confidenceintervalh = confidenceintervalh[1:]
    #print(confidenceintervalh)

    #Create x and y coordinates index
    coordinateh = np.arange(0, 63, 1).tolist()
    csi2 = set(confidenceintervalh)
    coorindexhand = [x for x in coordinateh if x not in csi2]
    #print(coorindexhand)

    person1 = []                                        #CHANGE VARIABLE NAME
    for framepose in p1:
        signer1 = pd.DataFrame(framepose) 
        hand1_L = signer1.iloc[1,:]
        hand1_R = signer1.iloc[2,:]
        hand1_L = hand1_L.dropna(axis='rows')
        hand1_R = hand1_R.dropna(axis='rows')
        #Get confidence intervals for signer 1 and 2 per keypoint per hand
        H1L = [hand1_L[i] for i in confidenceintervalh]
        H1R = [hand1_R[i] for i in confidenceintervalh]
        #Get x and y coordinates for signer 1 and 2 per keypoint
        hand1R = [hand1_L[i] for i in coorindexhand]
        hand1L = [hand1_R[i] for i in coorindexhand]
        #print(len(hand1L))
        #Split the list into x, y coordinate arrays
        handR11 = np.array_split(hand1R, 21)
        handL11 = np.array_split(hand1L, 21)
        #Normalize on 240, from Songha Ban's code.
        handR1 = scale(normalizer(handR11, 240))
        handL1 = scale(normalizer(handL11, 240))
        #Join them into 1 list with numpy arrays
        hand1 = [handR1, handL1]
        person1.append(hand1)

    person2 = []                                        #CHANGE VARIABLE NAME
    for framepose in p2:
        signer2 = pd.DataFrame(framepose) 
        hand2_L = signer2.iloc[1,:]
        hand2_R = signer2.iloc[2,:]
        hand2_L = hand2_L.dropna(axis='rows')
        hand2_R = hand2_R.dropna(axis='rows')
        #Get confidence intervals for signer 1 and 2 per keypoint per hand
        H2L = [hand2_R[i] for i in confidenceintervalh]
        H2R = [hand2_R[i] for i in confidenceintervalh]
        #Get x and y coordinates for signer 1 and 2 per keypoint
        hand2R = [hand2_L[i] for i in coorindexhand]
        hand2L = [hand2_R[i] for i in coorindexhand]
        #print(len(hand1L))
        #Split the list into x, y coordinate arrays
        handR22 = np.array_split(hand2R, 21)
        handL22 = np.array_split(hand2L, 21)
        handR2 = scale(handR22)
        handL2 = scale(handL22)
        #Join them into 1 list with numpy arrays
        hand2 = [handR2, handL2]
        person2.append(hand2)
    return person1, person2

def data_shaper(signers, hands):
    

def label_loader(labelspath):
    data = pd.read_csv(labelspath)
    data2 = data.iloc[:,3]
    labels = data2
    return [labels]

def finish_processing(path,fps,labelspath):
    framelist = framenames(path, fps)
    poses = data_loader(framelist)
    processedsk = pose_processor(poses)
    processedhands = hand_processor(poses)
    features = data_shaper(processedsk,processedhands)
    labels = label_loader(labelspath)
    return processedhands

#TO DO: COMBINE HANDS TO CORRECT SKELETON DATA AS FEATURES
#       RESHAPE DATA TO CORRECT FORM

In [31]:
path = 'D:\Videos thesis\KN5Jan7_poses\KN5Jan7_poses\\'
fps = 24
labelspath = 'D:\Videos thesis\KN5jan7_annotated.csv'
z = finish_processing(path, fps, labelspath)


There are 0 frames with 1 person in them
There are 809 frames with 2 persons in them
There are 90 frames with 3 persons in them
There are 1 frames with 4 persons in them
There are 0 frames with 5 persons in them
2


In [41]:
labelspath = 'D:\Videos thesis\KN5jan7_annotated.csv'

def label_loader(labelspath):
    data = pd.read_csv(labelspath)
    data2 = data.iloc[:,3]
    labels = data2
    return [labels]

z = label_loader(labelspath)
print(z)

[0      0
1      0
2      0
3      0
4      0
      ..
895    1
896    1
897    3
898    3
899    3
Name: class, Length: 900, dtype: int64]


In [ ]:
# FORMAT FOR LSTM INPUT
# [samples, time steps and features]

##                          ##
##        MODEL             ##
##                          ##


In [ ]:
#https://spyjetson.blogspot.com/2019/10/jetsonnano-human-pose-estimation-using.html

#SKELETON KEYPOINTS
{
//     {0,  "Nose"},
//     {1,  "Neck"},
//     {2,  "RShoulder"},
//     {3,  "RElbow"},
//     {4,  "RWrist"},
//     {5,  "LShoulder"},
//     {6,  "LElbow"},
//     {7,  "LWrist"},
//     {8,  "MidHip"},
//     {9,  "RHip"},
//     {10, "RKnee"},
//     {11, "RAnkle"},
//     {12, "LHip"},
//     {13, "LKnee"},
//     {14, "LAnkle"},
//     {15, "REye"},
//     {16, "LEye"},
//     {17, "REar"},
//     {18, "LEar"},
//     {19, "LBigToe"},
//     {20, "LSmallToe"},
//     {21, "LHeel"},
//     {22, "RBigToe"},
//     {23, "RSmallToe"},
//     {24, "RHeel"},
//     {25, "Background"}
// };

#HAND KEYPOINTS
#as seen on left hand when it is held open 
{
//     {0,  "BotR palm"},
//     {1,  "BotL palm"},
//     {2,  "Thumb Bot"},
//     {3,  "Thumb Mid"},
//     {4,  "Thumb Top"},
//     {5,  "Index Bot"},
//     {6,  "Index Mid1"},
//     {7,  "Index Mid2"},
//     {8,  "Index Top"},
//     {9,  "Middle Bot"},
//     {10, "Middle Mid1"},
//     {11, "Middle Mid2"},
//     {12, "Middle Top"},
//     {13, "Ring Bot"},
//     {14, "Ring Mid1"},
//     {15, "Ring Mid2"},
//     {16, "Ring Top"},
//     {17, "Pinky Bot"},
//     {18, "Pinky Mid1"},
//     {19, "Pinky Mid2"},
//     {20, "Pinky Top"},
// };




In [85]:
#skeleton12.to_excel("signer2.xlsx") 

In [7]:
#Openpose Skeleton
#https://cmu-perceptual-computing-lab.github.io/openpose/web/html/doc/md_doc_02_output.html
#https://github.com/SonghaBan/DancingAI/blob/main/visframe.py 

l_pair_openpose25 = [
    [1, 2], [1, 5], [2, 3], [3, 4], [5, 6], [6, 7], [1, 8], 
    [8, 9], [9, 10], [10,11], [11,24], [11,22], [22,23],
    [8,12], [12, 13], [13,14], [14,21], [14,19], [19,20],
    [1, 0], 
    [0, 16], [18, 16], 
    [0, 15], [15, 17],
    ]

colors_openpose25 = np.array([
    [255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0],
    [85, 255, 0], [255,0,0], 
    [0, 255, 0], [0, 255, 85], [0, 255, 85], [0, 255, 85], [0, 255, 85], [0, 255, 85], 
    [0, 255, 170], [0, 255, 255],[0, 170, 255],[0, 170, 255],[0, 170, 255],[0, 170, 255],
    [0, 85, 255], 
#    [0, 0, 255], [85, 0, 255], 
#    [170, 0, 255],
    [255, 0, 255], [255, 0, 170], 
    [255, 0, 85], [255, 0, 0]])/255

print(len(l_pair_openpose25))
print(len(colors_openpose25))

24
24


In [ ]:
#Visualize Frame
#https://github.com/SonghaBan/DancingAI/blob/main/visframe.py 

#import torch
#import math
#import cv2
#import time
#from more_itertools.recipes import grouper, pairwise
#import moviepy.editor as mpy
#import more_itertools



def show_img(arr,idx=0):
    img = Image.fromarray(arr)
#    img.show()
    img.save('test1.jpg')

def visualize_frame(frame, joints=17, label=0):
    
    surface = gz.Surface(width=640, height=360, bg_color=(1,1,1))
    
    frame = open(frame)
    frame = json.load(frame)
    #frame.close()

    if joints == 25:
        frame = frame['people'][0]
        n_group = int(len(frame['pose_keypoints_2d']) / joints)
        pose = list(grouper(frame["pose_keypoints_2d"], n_group))
    else:
        n_group = int(len(frame['keypoints']) / joints)
        pose = list(grouper(frame["keypoints"], n_group))
    print(pose)
    
    if joints==25:
        l_pair = l_pair_openpose25
        line_color = colors_openpose25
    elif joints == 26:
        l_pair = l_pair_26
        line_color = line_color_26
        
    line_cnt = 0
    
    for limb in l_pair:
    #    print(limb)
        if limb[0] == joints and (joints == 17 or joints == 26):
            x1, y1, _ = (np.array(pose[5]) + np.array(pose[6])) / 2 #neck
        else:
            x1, y1, _ = pose[limb[0]]
        x2, y2, _ = pose[limb[1]]
        line = gz.polyline(points=[(x1,y1), (x2,y2)], stroke_width = 6, stroke=line_color[line_cnt])
    #    print('line', line_cnt)
        line_cnt += 1
        line.draw(surface)
    
    for idx in range(len(pose)):
    #    print(idx)
        x1, y1,_ = pose[idx]
        joint = gz.circle(4, xy=[x1,y1], fill=(0,0,0))
        joint.draw(surface)

    show_img(surface.get_npimage(), label)



In [ ]:
visualize_frame('D:\Videos thesis\SuJu16_poses\SuJu16_poses\\frame7_keypoints.json', joints = 25)

In [ ]:
#POST PROCESSING

#check if difference between frames is too large


#https://github.com/SonghaBan/DancingAI


#def check_previous_frame(filename)

#compute difference with one frame before here


In [ ]:
#https://sduxbury.medium.com/how-you-can-build-practical-applications-by-quantifying-observations-from-video-e266b945eea0
#Point sets are in the form [coordinate, coordinate, confidence]